<a href="https://colab.research.google.com/github/mcagriaksoy/TreeSegmentation/blob/main/SegFormerTransformator_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#####################################################
# Hello! Welcome to my playground! It is Mehmet Cagri Aksoy - 2023
# more information about me: https://github.com/mcagriaksoy
# This notebook is created for educational and academic purposes.
# References:
# https://keras.io/examples/vision/segformer/
#####################################################

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#Get Sample Dataset
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incompleteBOJFS2/oxford_iiit_pet-train.tfrecord*...:…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incompleteBOJFS2/oxford_iiit_pet-test.tfrecord*...: …

Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


In [85]:
#We NORMALIZE the image data for better performance and get rid of ugly numbers :D
from tensorflow.keras import backend

mean = tf.constant([0.485, 0.456, 0.406])
std = tf.constant([0.229, 0.224, 0.225])

def normalize(input_image, input_mask):
    input_image = tf.image.convert_image_dtype(input_image, tf.float32)
    input_image = (input_image - mean) / tf.maximum(std, backend.epsilon())
    input_mask -= 1
    return input_image, input_mask

In [95]:
# load images from dataset
# resized to 128x128
image_size = 128
def load_image(datapoint):
    input_image = tf.image.resize(datapoint["image"], (image_size, image_size))
    input_mask = tf.image.resize(
        datapoint["segmentation_mask"],
        (image_size, image_size),
        method="bilinear",
    )

    input_image, input_mask = normalize(input_image, input_mask)
    input_image = tf.transpose(input_image, (2, 0, 1))
    return {"pixel_values": input_image, "labels": tf.squeeze(input_mask)}

In [97]:
# Model definitions
BATCH_SIZE = 4
BUFFER_SIZE = BATCH_SIZE * 10

train_batches = (
     dataset["train"]
    .cache()
    .shuffle(BUFFER_SIZE)
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE))

validation_batches = (
    dataset["test"]
    .take(3000)
    .batch(BATCH_SIZE))

test_batches = (
    dataset["test"]
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE))

In [98]:
test_batches.element_spec

{'pixel_values': TensorSpec(shape=(None, 3, 128, 128), dtype=tf.float32, name=None),
 'labels': TensorSpec(shape=(None, 128, 128), dtype=tf.float32, name=None)}

In [ ]:
# You have select one of models: You cannot use both!

# segFormer Transformator Model

In [22]:
!!pip install transformers -q

[]

In [69]:
from transformers import TFSegformerForSemanticSegmentation

model_checkpoint = "nvidia/mit-b0"
id2label = {0: "outer", 1: "inner", 2: "border"}
label2id = {label: id for id, label in id2label.items()}
num_labels = len(id2label)
model = TFSegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
)

model.summary()

Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_segformer_for_semantic_segmentation_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 segformer (TFSegformerMainL  multiple                 3319392   
 ayer)                                                           
                                                                 
 decode_head (TFSegformerDec  multiple                 396035    
 odeHead)                                                        
                                                                 
Total params: 3,715,427
Trainable params: 3,714,915
Non-trainable params: 512
_________________________________________________________________


# Compile & Train the model

In [74]:
# Compiling the model!
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=None)

In [99]:
# Training the model!
NUM_EPOCHS = 10

model_history = model.fit(train_batches,
                              epochs=NUM_EPOCHS,
                              validation_data=test_batches)

Epoch 1/10


TypeError: ignored

In [16]:
#Prediction Part and tataaaaa!

def create_mask(pred_mask):
 pred_mask = tf.argmax(pred_mask, axis=-1)
 pred_mask = pred_mask[..., tf.newaxis]
 return pred_mask[0]
def show_predictions(dataset=None, num=1):
 if dataset:
   for image, mask in dataset.take(num):
     pred_mask = model.predict(image)
     display([image[0], mask[0], create_mask(pred_mask)])
 else:
   display([sample_image, sample_mask,
            create_mask(model.predict(sample_image[tf.newaxis, ...]))])
count = 0
for i in test_batches:
   count +=1
print("number of batches:", count)

number of batches: 11
